# Sudoku

I recently started solving sudoku puzzles for fun,
and the thought occured to me: 
  
    Why not try to write a sudoku solver in Python?
    I mean, how hard can it be?

## No backtracking, please...

A quick google search for how to solve sudokus programmatically
will give you many *backtracking* solutions. While *backtracking*
is a great way to solve sudokus, I personally just don't like it.
Backtracking can be summarized as follows: *

    Guess random numbers until the sudoku is solved.
    If an incorrect number has been guessed, then backtrack and
    guess a different number. Repeat until the sudoku is solved.
    
Backtracking not only lacks elegance,
but, from an *algorithmic* perspective, it's not very efficient.
While 9x9 sudokus can be solved rather quickly using backtracking,
this approach would not scale well to larger sudokus.

I want to solve a sudoku differently; I wanted to come up with an algorithm
that more closely matches how one might solve a sudoku in real life: by using
logic!

## Graph processing approach

When I solve a sudoku I am constantly thinking
about how all of the cells are interconnected.
The value of one cell effects the possible values of other cells,
which in turn effects other cells, and so on.
These cells can be said to be *connected* to other cells,
and what better way of modeling complex connections than
*graph processing*.

Cells can be connected to each other either by
being in the same *row*, *column*, or *box*.

The purpose of this tutorial is to see if we can use *graph-processing*
to solve a sudoku. It's also an exploration in
creating *expressive algorithms*: algorithms that mimick
the same logic we would use to solve sudokus in real life.

```{note}
This tutorial is limited to 9x9 sudokus.
It should technically be possible to generalize
this solution and make it work with other sizes of sudokus,
but that's outside the scope of this article.
```


## Sudoku data structure

As mentioned previously, we will model our sudoku as a *graph*.
We will use [networkx](https://networkx.org/documentation/stable/index.html)
for its wonderful *graph* functionalities.

We represent each *cell* of the sudoku as a *node*.
The nodes be equal to the index of each cell. Index `(0,0)`
is the top-left cell, and index `(8,8)` is the bottom-right cell.

We represent the *connections* between nodes by *edges*.
Cells can be connected by being in the same *row*, *edge*, or *box*.

Since cells can be connected to one another in multiple ways
(e.g. `(0,0)` is connectod to `(0,1)` by being in the same row
and in the same box), we will use a `MultiGraph` to model our
sudoku.

```{note}
Many graph-processing algorithms don't work correctly
with a MultiGraph. This is something to keep in mind.
Luckily we can easily convert our graph to a Graph.
See the [MultiGraph documentation](https://networkx.org/documentation/stable/tutorial.html#multigraphs)
for more details.
```

In [ ]:
import itertools
import networkx as nx
import matplotlib.pyplot as plt
from typing import List, Dict, Union, Callable

Sudoku = nx.MultiGraph

def gen_sudoku()->Sudoku:
    n=3
    dim = n*n
    sudoku = Sudoku()
    sudoku.graph["possible_values"] = set(range(1, 10))
    
    # Create a node for each cell in the sudoku
    for row, col in itertools.product(range(dim), range(dim)):
        # Each node is the row-column, and has a 'value'
        # We set the value to '0' initially
        sudoku.add_node((row,col), value=0)
        
    # Create the edges for each combination of nodes
    sudoku.graph["connections"] = ["row", "column", "box"]
    for n1, n2 in itertools.combinations(sudoku.nodes, r=2):
        n1_row, n1_col = n1
        n2_row, n2_col = n2
        n1_box = [i // n for i in [n1_row, n1_col]]
        n2_box = [i // n for i in [n2_row, n2_col]]
        
        if n1_row == n2_row:
            sudoku.add_edge(n1, n2, connection="row")
        if n1_col == n2_col:
            sudoku.add_edge(n1, n2, connection="column")
        if n1_box == n2_box:
            sudoku.add_edge(n1, n2, connection="box")
        
    return sudoku

sudoku = gen_sudoku()

## Visualizing the sudoku

Now that we have a sudoku data structure,
let's visualize it.

In [ ]:
def plot_sudoku(sudoku)->None:
    # - node (0,0) is at pos (0,8)
    # - node (0,1) is at pos (1,8)
    # - node (8,8) is at pos(8,0)
    pos = {n: (n[1], 8 - n[0]) for n in sudoku.nodes}
    # we label the nodes with their values
    labels = {n:sudoku.nodes[n]["value"] for n in sudoku.nodes}
    # the color of each node will be based on its value
    node_color = [v for v in labels.values()]
    nx.draw(
        sudoku, 
        pos=pos, 
        labels=labels, 
        with_labels=True, 
        font_color="black", 
        node_color=node_color,
        cmap=plt.cm.Set3
    )
    low = -0.5
    high = 8.5
    lines = [low, 2.5, 5.5, high]
    for l in lines:
        plt.plot([l,l], [low, high], color="g")
        plt.plot([low, high], [l,l], color="g")
    plt.show()
    
plot_sudoku(sudoku)

## Understanding the complex connections

Remember that the whole point of using a *graph data structure*
was to represent the *connections* between the nodes.
As mentioned earlier, graph-processing algorithms work
on graphs that are not multi-graphs.
We will be taking our multi-graph, which has
nodes connected in multiple ways, and split it
up into individual graphs whose nodes are only connected in a single way.

In [ ]:
def get_connection_sudoku_map(sudoku:Sudoku)->dict[str, nx.Graph]:
    all_nodes = list(sudoku.nodes(data=True))
    all_edges = [(u, v, c) for (u, v, c) in sudoku.edges(data="connection")]
    
    sudoku_connections: SudokuConnections = {}
    
    for conn in sudoku.graph["connections"]:
        sc = nx.Graph()
        sc.add_nodes_from(all_nodes)
        edges_to_keep = [(u, v) for u, v, c in all_edges if c == conn]
        sc.add_edges_from(edges_to_keep)
        sudoku_connections[conn] = sc
        
    sc_all = nx.Graph()
    sc_all.add_nodes_from(all_nodes)
    sc_all.add_edges_from([u, v] for u, v, c in all_edges)
    sudoku_connections["all"] = sc_all
    
    return sudoku_connections
    

def plot_connections(sudoku_connections):
    for conn, sc in sudoku_connections.items():
        plt.title(conn)
        plot_sudoku(sc)
    
    
sudoku_connections = get_connection_sudoku_map(sudoku)
plot_connections(sudoku_connections)

The *row*, *column*, and *box* connections make sense to you,
but you're probably wondering about the *all* connection...
What this represents is *all* of the nodes an individual node
is connected to; the *all* connection encompasses the *row-column-box*
connection.

### Connected components

Take a look at the *row* plot above. You'll see that
one row is not connected to any other row.
An individual row, in In graph terminology, constitutes
a *connected component*. A connected component is a collection
of nodes that are connected to one another. In our case,
there are as many *connected components* in the *row graph* as there are rows.
The same is true for all of the different *connection graphs* above.

In [ ]:
first_row_nodes = list(nx.connected_components(sudoku_connections["row"]))[0]
plot_sudoku(sudoku.subgraph(first_row_nodes))

I won't plot al of the connected components of all
of the different connections, but you get the point.

### Node neighbors

The neighbors of a node are the nodes that it is connected to.
Let's take a look at all the different ways a node is connected to other nodes.

In [ ]:
def plot_sudoku_connection_neighbors(sudoku, node):
    sudoku_connection = get_connection_sudoku_map(sudoku)
    sudoku_connections_neighbors = {}
    for conn, sc in sudoku_connections.items():
        neighbors = sc.neighbors(node)
        sub = sc.subgraph(neighbors)
        sudoku_connections_neighbors[conn] = sub
        
    plot_connections(sudoku_connections_neighbors)
    
plot_sudoku_connection_neighbors(sudoku, (4,4))

## Creating a *solver*

Now we're ready to get started on solving a sudoku!
First, we need a way to tell if a sudoku is  actually solved.

In [ ]:
def is_sudoku_solved(sudoku:Sudoku)->bool:
    # a sudoku is unsolved if there are any zeros
    if any([v == 0 for n, v in sudoku.nodes(data="value")]):
        return False
    # a sudoku is unsolved if any 'connections' don't have all numbers 1-9
    sudoku_connections = get_connection_sudoku_map(sudoku)
    for conn, sc in sudoku_connections.items():
        for cc in nx.connected_components(sc):
            values = set(sudoku.nodes[n]["value"] for n in cc)
            if len(values) != len(sudoku.graph["possible_values"]):
                return False
    # if we made it this far, then sudoku is solved
    return True


assert not is_sudoku_solved(sudoku)

Now let's create a function for solving a sudoku.
Remember that we want our solver to follow the same
logical steps that we would take for solving a sudoku:

1. We start with an unsolved sudoku
2. We use a *technique* to solve the value for cells without a value
3. If our technique worked, and we know the value of one or more cells, then we update the value of those cells
4. If our technique did not work, we try to use a more advanced technique
5. If all of our techniques did not work... then we admit defeat!

Here's what that might look like in code:

In [ ]:
def solve_sudoku(sudoku, *techniques)->Sudoku:
    # let's copy the original sudoku
    solved_sudoku = sudoku.copy()
    
    while not is_sudoku_solved(solved_sudoku):
        # we try one technique at a time
        for technique in techniques:
            # if a technique works
            if nodes_to_update:= technique(solved_sudoku):
                # then we update the nodes
                for n, val in nodes_to_update.items():
                    # only one caveat, the node we are trying
                    # to update must have a value of '0'
                    if not solved_sudoku.nodes[n]["value"] == 0:
                        raise ValueError(
                            "Trying to set value of node that's already set",
                            solved_sudoku,
                            n,
                        )
                    solved_sudoku.nodes[n]["value"] = val
                # breaking means that we simply start this
                # 'for-loop' again, starting from the first technique
                break    
        else:
            raise RuntimeError("Cannot solve sudoku", solved_sudoku)
    return solved_sudoku
        

def try_solve_sudoku(sudoku, *rules)->Sudoku:
    try:
        return solve_sudoku(sudoku, *rules)
    except RuntimeError as exc:
        msg, partially_solved_sudoku = exc.args
        print(msg)
        return partially_solved_sudoku
    except ValueError as ecx:
        msg, partially_solved_sudoku, node = exc.args
        print(msg, node)
        return partially_solved_sudoku

You'll see that we created two solving functions:
one that *tries to solve a sudoku*, and another that
will raise an exception if it cannot be solved.
For the sake of this tutorial, this was probably completely unnecessary...

Let's create our first technique.
This is just a silly technique of applying a value
to a node based on its position... it will only
work for a our sudoku where all values are `0`.

In [ ]:
def ascending_values_technique(sudoku):
    nodes_to_update = {}
    for n in sudoku.nodes:
        r, c = n
        br = r // 3
        val = (r*3 + c + 1 + br) % 9 or 9
        nodes_to_update[n] = val
    return nodes_to_update

sudoku_solved_by_ascending_technique = solve_sudoku(
    sudoku,
    ascending_values_technique
)
plot_sudoku(sudoku_solved_by_ascending_technique)

assert is_sudoku_solved(sudoku_solved_by_ascending_technique)

So far we have laid the groundwork for solving sudokus,
but we literally haven't solved anything yet!
Sadly, there are still two things we need to do:

1. Create a helper function for  creating sudokus with real values
2. Implement real techniques for solving real sudokus

## Creating real sudokus

In [ ]:
def gen_sudoku_with_values(values):
    sudoku = gen_sudoku()
    for i, row in enumerate(values):
        for j, val in enumerate(row):
            sudoku.nodes[(i,j)]["value"] = val
    return sudoku
            
easy_sudoku = gen_sudoku_with_values([
    [4,0,0,2,0,5,8,6,0],
    [0,0,0,0,0,8,4,9,3],
    [6,0,0,0,0,0,0,2,7],
    [1,0,0,0,6,9,0,5,0],
    [3,0,0,0,8,0,0,0,9],
    [0,5,0,4,3,0,0,0,2],
    [8,6,0,0,0,0,0,0,4],
    [7,2,1,8,0,0,0,0,0],
    [0,9,4,1,0,3,0,0,6]
])
plot_sudoku(easy_sudoku)

## Techniques for solving sudokus

Okay, we're finally ready to solve some real sudokus!
That felt like way too long of an intro... sorry.

Anyway, for a great resource on real sudoku-solving techniques
I referred to [this site](https://www.kristanix.com/sudokuepic/sudoku-solving-techniques.php). This was really helpful in giving me the vocabulary I needed
to describe the techniques I was already using,
and it also gave me ideas for new techniques.

### Getting the candidates of each node

One of the first things you do in trying to solve a sudoku
is to figure out what the possible values of a cell are.
If any cell can only be one value, then it must be that value!

In [ ]:
def get_node_candidates_map(sudoku:Sudoku)->dict[tuple[int, int], set]:
    node_candidates_map = {}
    sudoku_possible_values = sudoku.graph["possible_values"].copy()
    sudoku_connections = get_connection_sudoku_map(sudoku)
    
    for node, value in sudoku.nodes(data="value"):
        if value == 0:
            all_neighbors = list(sudoku_connections["all"].neighbors(node))
            values_of_all_neighbors = set(sudoku.nodes[n]["value"] for n in all_neighbors)
            values_of_all_neighbors.discard(0)
            node_candidates_map[node] = sudoku_possible_values - values_of_all_neighbors
        else:
            node_candidates_map[node] = {value}
    
    return node_candidates_map

def unique_candidate_technique(sudoku):
    updates = {}
    node_candidates_map = get_node_candidates_map(sudoku)
    for node, candidates in node_candidates_map.items():
        if sudoku.nodes[node]["value"] == 0 and len(candidates) == 1:
            updates[node] = list(candidates)[0]
            
    return updates
            
easy_solved = try_solve_sudoku(easy_sudoku, unique_candidate_technique)
plot_sudoku(easy_solved)

In [ ]:
def get_sudoku_impossible_values(sudoku:Sudoku):
    sudoku_impossibilities = {n:set() for n in sudoku.nodes}
    possible_values = sudoku.graph["possible_values"].copy()
    sudoku_connections = get_connection_sudoku_map(sudoku)
    
    for node, value in sudoku.nodes(data="value"):
        if value == 0:
            sc_all = sudoku_connections["all"]
            values_of_all_neighobrs = set([sudoku.nodes[n]["value"] for n in sc_all.neighbors(node)])
            values_of_all_neighobrs.discard(0)
            sudoku_impossibilities[node] = values_of_all_neighobrs
        else:
            all_other_values = possible_values - {value}
            sudoku_impossibilities[node] = all_other_values
            
    return sudoku_impossibilities

def solve_by_deducing_impossible_values(sudoku):
    impossible_values = get_sudoku_impossible_values(sudoku)
    possible_values = sudoku.graph["possible_values"]
    updates = {}
    for node, values in impossible_values.items():
        if sudoku.nodes[node]["value"] == 0 and len(values) == 8:
            value = list((possible_values - values))[0]
            updates[node] = value
    return updates
            
plot_sudoku(try_solve_sudoku(easy_sudoku, solve_by_deducing_impossible_values))

In [ ]:
%timeit try_solve_sudoku(easy_sudoku, solve_by_deducing_impossible_values)

In [ ]:
medium_sudoku = gen_sudoku_with_values([
    [9,0,0,3,0,0,0,0,0],
    [0,0,1,0,4,0,0,5,0],
    [8,0,6,0,0,1,0,4,0],
    [3,0,0,0,0,0,2,0,0],
    [1,0,5,4,3,2,7,0,9],
    [0,0,7,0,0,0,0,0,4],
    [0,1,0,5,0,0,4,0,6],
    [0,7,0,0,2,0,1,0,0],
    [0,0,0,0,0,7,0,0,8],
])
plot_sudoku(medium_sudoku)
medium_sudoku_solved = try_solve_sudoku(medium_sudoku, solve_by_deducing_impossible_values)
plot_sudoku(medium_sudoku_solved)
is_sudoku_solved(medium_sudoku_solved)

In [ ]:
def deduce_value_from_impossibilities(sudoku):
    possible_values = sudoku.graph["possible_values"]
    impossible_values = get_sudoku_impossible_values(sudoku)
    sudoku_connections = get_sudoku_connections(sudoku)
    updates = {}
    for node, value in sudoku.nodes(data="value"):
        if value == 0:
            for conn, sc in sudoku_connections.items():
                neighbors = sc.neighbors(node)
                imp_vals_of_neighbors = possible_values.copy()
                imp_vals_of_neighbors.intersection_update(
                    *[impossible_values[n] for n in neighbors]
                )
                if len(imp_vals_of_neighbors) == 1:
                    updates[node] = list(imp_vals_of_neighbors)[0]
    return updates
            
plot_sudoku(try_solve_sudoku(medium_sudoku, deduce_value_from_impossibilities))

In [ ]:
hard_sudoku = gen_sudoku_with_values([
    [2,0,0,0,6,0,7,3,0],
    [0,0,9,0,2,3,0,8,6],
    [0,0,0,5,0,8,1,0,0],
    [9,5,0,0,0,0,0,1,0],
    [8,0,6,0,3,0,5,0,9],
    [0,4,0,0,0,0,0,7,8],
    [0,0,2,3,0,5,0,0,0],
    [4,1,0,2,9,0,3,0,0],
    [0,6,5,0,8,0,0,0,1],
])
plot_sudoku(hard_sudoku)
hard_sudoku_solved = try_solve_sudoku(hard_sudoku, deduce_value_from_impossibilities)
plot_sudoku(hard_sudoku_solved)
is_sudoku_solved(hard_sudoku_solved)

In [ ]:
evil_sudoku = gen_sudoku_with_values([
    [0,1,0,0,0,8,0,0,3],
    [0,0,0,9,0,1,0,2,8],
    [0,0,4,0,0,0,0,0,0],
    [0,0,1,0,0,0,0,3,4],
    [0,3,0,0,6,0,0,9,0],
    [2,5,0,0,0,0,7,0,0],
    [0,0,0,0,0,0,3,0,0],
    [6,4,0,5,0,9,0,0,0],
    [9,0,0,8,0,0,0,1,0],
])
plot_sudoku(evil_sudoku)
evil_sudoku_solved = try_solve_sudoku(
    evil_sudoku,
    solve_by_deducing_impossible_values,
    deduce_value_from_impossibilities
)
plot_sudoku(evil_sudoku_solved)
print(is_sudoku_solved(evil_sudoku_solved))
